In [47]:
import os
import pandas as pd
import numpy as np
import librosa
import glob
import matplotlib.pyplot as plt
import sox
import scipy
import scipy.io.wavfile
import sys
import glob
import sklearn 
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.externals import joblib
from random import shuffle
from sklearn.naive_bayes import GaussianNB
from python_speech_features import mfcc

In [48]:
# Directory where the music dataset is located (GTZAN dataset)
GENRE_DIR = "/Users/rtz/Desktop/Github/Genre-classification/Genre-classification/genres/"

# Directory where the test music is located
TEST_DIR = "/Users/rtz/Desktop/Github/Genre-classification/Genre-classification/test_genres/"

# All the available genres
GENRE_LIST = [ "blues","classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"]

In [49]:
'''
For converting the extension from "au" to "wav"
genre_directory = ["/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/blues",
                   "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/classical",
                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/country/",
                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/disco/",
                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/hiphop/",
                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/jazz/",
                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/metal/",
                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/pop/",
                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/reggae/",
                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/rock/"]
for genre in genre_directory:
    os.chdir(genre)
    for file in os.listdir(genre):
        # SOX
        os.system("sox " + str(file) + " " + str(file[:-3]) + ".wav")
    # delete .au from current dir
    os.system("rm *.au")
'''

'\nFor converting the extension from "au" to "wav"\ngenre_directory = ["/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/blues",\n                   "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/classical",\n                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/country/",\n                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/disco/",\n                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/hiphop/",\n                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/jazz/",\n                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/metal/",\n                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/pop/",\n                  "/Users/rtz/Desktop/GitHub/Genre-classification/Genre-classification/genres/reggae/",\n       

In [50]:
# Extracts frequencies from a wavile and stores in a file
def create_fft(wavfile): 
    sample_rate, song_array = scipy.io.wavfile.read(wavfile)
    #print(sample_rate)
    fft_features = abs(scipy.fft(song_array[:30000]))
    #print(song_array)
    base_fn, ext = os.path.splitext(wavfile)
    data_fn = base_fn + ".fft"
    np.save(data_fn, fft_features)
def read_fft(genre_list, base_dir):
    X = []
    y = []
    for label, genre in enumerate(genre_list):
        # create UNIX pathnames to id FFT-files.
        genre_dir = os.path.join(base_dir, genre, "*.fft.npy")
        # get path names that math genre-dir
        file_list = glob.glob(genre_dir)
        for file in file_list:
            fft_features = np.load(file)
            X.append(fft_features)
            y.append(label)
    
    return np.array(X), np.array(y)




In [55]:
def create_ceps(wavfile):
    sampling_rate, song_array = scipy.io.wavfile.read(wavfile)
    #print(sampling_rate)
    """Get MFCC
    ceps  : ndarray of MFCC
    mspec : ndarray of log-spectrum in the mel-domain
    spec  : spectrum magnitude
    """
    ceps=mfcc(song_array)
    #ceps, mspec, spec= mfcc(song_array)
    #print(ceps.shape)
    #this is done in order to replace NaN and infinite value in array
    bad_indices = np.where(np.isnan(ceps))
    b=np.where(np.isinf(ceps))
    ceps[bad_indices]=0
    ceps[b]=0
    write_ceps(ceps, wavfile)
    
"""reads MFCC-files and prepares X_train and y_train.
genre_list must consist of names of folders/genres consisting of the required MFCC-files
base_dir must contain genre_list of directories
"""
def write_ceps(ceps, wavfile):
    base_wav, ext = os.path.splitext(wavfile)
    data_wav = base_wav + ".ceps"
    np.save(data_wav, ceps)
    
def read_ceps(genre_list, base_dir):
    X= []
    y=[]
    for label, genre in enumerate(genre_list):
        for fn in glob.glob(os.path.join(base_dir, genre, "*.ceps.npy")):
            ceps = np.load(fn)
            num_ceps = len(ceps)
            X.append(np.mean(ceps[int(num_ceps*1/10):int(num_ceps*9/10)], axis=0))
            #X.append(ceps)
            y.append(label)
    return X,y

In [56]:
def learn_and_classify(X_train, y_train, X_test, y_test, genre_list):

    #Logistic Regression classifier

    logistic_classifier = linear_model.logistic.LogisticRegression()
    logistic_classifier.fit(X_train, y_train)
    logistic_predictions = logistic_classifier.predict(X_test)
    logistic_accuracy = accuracy_score(y_test, logistic_predictions)
    logistic_cm = confusion_matrix(y_test, logistic_predictions)
    print("logistic accuracy = " + str(logistic_accuracy))
    #print("logistic_cm:")
    #print(logistic_cm *100)


    #K-Nearest neighbour classifier

    knn_classifier = KNeighborsClassifier()
    knn_classifier.fit(X_train, y_train)
    knn_predictions = knn_classifier.predict(X_test)
    knn_accuracy = accuracy_score(y_test, knn_predictions)
    knn_cm = confusion_matrix(y_test, knn_predictions)
    print("knn accuracy = " + str(knn_accuracy))
    #print("knn_cm:") 
    #print(knn_cm * 100)
    
    #Naive Bayes classifier
    gnb = GaussianNB()
    y_pred = gnb.fit(X_train,y_train).predict(X_test)
    gnb_accuracy = accuracy_score(y_test, y_pred)
    print("Naive Bayes accuracy = " + str(gnb_accuracy))
    gnb_cm = confusion_matrix(y_test, y_pred)
    #print("knn_cm:") 
    #print(knn_cm * 100)
    
    plot_confusion_matrix(logistic_cm, "Confusion matrix for logistic regression", genre_list)
    plot_confusion_matrix(knn_cm, "Confusion matrix for KNN", genre_list)
    plot_confusion_matrix(gnb_cm, "Confusion matrix for Naive Bayes", genre_list)


def plot_confusion_matrix(cm, title, genre_list, cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(genre_list))
    plt.xticks(tick_marks, genre_list, rotation=45)
    plt.yticks(tick_marks, genre_list)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
def main():

    for label, genre in enumerate(GENRE_LIST):
        for fn in glob.glob(os.path.join(GENRE_DIR, genre)):
            for wavfile in os.listdir(fn):
                    if wavfile.endswith("wav"):
                        create_fft(os.path.join(GENRE_DIR, genre,wavfile))
    print(GENRE_DIR)
    base_dir_fft  = GENRE_DIR
    base_dir_mfcc = GENRE_DIR
    
    """list of genres (these must be folder names consisting .wav of respective genre in the base_dir)
    Change list if needed.
    """
    genre_list = [ "blues","classical","country","disco","metal"]
    
    #genre_list = ["classical", "jazz"] IF YOU WANT TO CLASSIFY ONLY CLASSICAL AND JAZZ

    '''use FFT
    X, y = read_fft(genre_list, base_dir_fft)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20)
    print('\n******USING FFT******')
    learn_and_classify(X_train, y_train, X_test, y_test, genre_list)
    print('*********************\n')
    '''
    #use MFCC
    for label, genre in enumerate(GENRE_LIST):
        for fn in glob.glob(os.path.join(GENRE_DIR, genre)):
            for wavfile in os.listdir(fn):
                if wavfile.endswith("wav"):
                    create_ceps(os.path.join(GENRE_DIR, genre,wavfile))
    X,y= read_ceps(genre_list, base_dir_mfcc)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20)
    print("new1",X_train.shape)
    print('******USING MFCC******')
    learn_and_classify(X_train, y_train, X_test, y_test, genre_list)
    print('*********************')
if __name__ == "__main__":
    main()

/Users/rtz/Desktop/Github/Genre-classification/Genre-classification/genres/


AttributeError: 'list' object has no attribute 'shape'